In [2]:
!pip install langchain langchain_community langgraph langchain_groq duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 19.8 MB/s eta 0:00:00


In [3]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from duckduckgo_search import DDGS
import re
import os
from google.colab import userdata

In [4]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


In [5]:
def search_historical_info(query: str) -> str:
    """Searches for historical information using DuckDuckGo.
    Use this tool for any historical queries or date-related questions."""
    try:
        search = DDGS()
        results = search.text(query, max_results=3)

        if results:
            # Look for years in the results
            for result in results:
                text = result.get('body', '') + ' ' + result.get('title', '')
                # Look for 4-digit years
                years = re.findall(r'\b(19\d{2}|20\d{2})\b', text)
                if years:
                    return f"Found historical information: {text[:200]}... Relevant years found: {years}"

            # If no years found, return the first result
            return f"Historical information: {results[0].get('body', '')[:300]}"
        else:
            return "No historical information found for this query."
    except Exception as e:
        return f"Search error: {str(e)}"

In [6]:
def perform_multiplication(number1: int, number2: int) -> int:
    """Multiplies two numbers together.
    ALWAYS use this tool for ANY multiplication operation - do not calculate manually."""
    result = number1 * number2
    print(f"🔢 Multiplication Tool: {number1} × {number2} = {result}")
    return result

def perform_addition(number1: int, number2: int) -> int:
    """Adds two numbers together.
    ALWAYS use this tool for ANY addition operation - do not calculate manually."""
    result = number1 + number2
    print(f"➕ Addition Tool: {number1} + {number2} = {result}")
    return result

def perform_subtraction(number1: int, number2: int) -> int:
    """Subtracts the second number from the first number.
    ALWAYS use this tool for ANY subtraction operation - do not calculate manually."""
    result = number1 - number2
    print(f"➖ Subtraction Tool: {number1} - {number2} = {result}")
    return result


In [7]:
def get_current_year() -> int:
    """Gets the current year. Use this when you need to know what year it is now."""
    from datetime import datetime
    current_year = datetime.now().year
    print(f"📅 Current Year Tool: {current_year}")
    return current_year

def calculate_age_from_year(birth_year: int) -> int:
    """Calculates how many years ago something happened or someone's age from their birth year."""
    from datetime import datetime
    current_year = datetime.now().year
    age = current_year - birth_year
    print(f"🎂 Age Calculator Tool: {current_year} - {birth_year} = {age} years")
    return age


In [8]:
os.environ["groq_Api"] = userdata.get("groq_Api")

In [9]:
llm = ChatGroq(
    groq_api_key=os.environ["groq_Api"],
    model_name="llama3-8b-8192",
    temperature=0  # Lower temperature for more consistent tool usage
)

In [10]:
tools = [
    search_historical_info,
    perform_multiplication,
    perform_addition,
    perform_subtraction,
    get_current_year,
    calculate_age_from_year
]

In [11]:
llm_with_tools = llm.bind_tools(tools)

In [12]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state['messages'])]}


In [13]:
# Build the graph
graph_builder = StateGraph(State)
graph_builder.add_node("assistant", chatbot)
graph_builder.add_node("tools", ToolNode(tools))

graph_builder.add_edge(START, "assistant")
graph_builder.add_conditional_edges("assistant", tools_condition)
graph_builder.add_edge("tools", "assistant")

react_graph = graph_builder.compile()

In [14]:
# Enhanced system message that forces tool usage
SYSTEM_MESSAGE = """You are a helpful assistant with access to specialized tools.

IMPORTANT RULES:
1. You MUST use the provided tools for ALL calculations - never perform math manually
2. For ANY arithmetic operation (addition, subtraction, multiplication), use the corresponding tool
3. For historical information or dates, use the search_historical_info tool
4. Always use tools even for simple calculations that you could do yourself
5. If you need the current year, use the get_current_year tool
6. Show your work by using tools step by step

Available tools:
- search_historical_info: For historical queries and dates
- perform_multiplication: For multiplication (REQUIRED for all multiplication)
- perform_addition: For addition (REQUIRED for all addition)
- perform_subtraction: For subtraction (REQUIRED for all subtraction)
- get_current_year: To get the current year
- calculate_age_from_year: To calculate years since an event

Remember: You must demonstrate the use of these tools rather than doing calculations in your head."""


In [15]:
# Test cases
test_queries = [
    "What year did India gain independence? Then multiply that year by 2 and add 5.",
    "What year was the moon landing? Calculate how many years ago that was.",
    "Find the year World War 2 ended. Then subtract 1900 from that year and multiply the result by 3.",
    "What's 25 multiplied by 4, then add 17 to the result?"
]

In [16]:
print("🚀 Testing Enhanced Tool Usage\n")
print("=" * 60)


🚀 Testing Enhanced Tool Usage



In [17]:
for i, query in enumerate(test_queries, 1):
    print(f"\n🔍 Test {i}: {query}")
    print("-" * 50)

    response = react_graph.invoke({
        "messages": [
            SystemMessage(content=SYSTEM_MESSAGE),
            HumanMessage(content=query)
        ]
    })


🔍 Test 1: What year did India gain independence? Then multiply that year by 2 and add 5.
--------------------------------------------------


/tmp/ipython-input-133929757.py:5: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  search = DDGS()
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e05917809f0>
/tmp/ipython-input-133929757.py:5: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  search = DDGS()
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e05917810f0>


🔢 Multiplication Tool: 1947 × 2 = 3894


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e05917812b0>


➕ Addition Tool: 3894 + 5 = 3899


...

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e0591780360>



🔍 Test 4: What's 25 multiplied by 4, then add 17 to the result?
--------------------------------------------------
🔢 Multiplication Tool: 25 × 4 = 100


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e05917810f0>


➕ Addition Tool: 100 + 17 = 117


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e0591782190>


In [18]:
  # print("\n📋 Conversation Flow:")


📋 Conversation Flow:


In [21]:
for j, message in enumerate(response["messages"]):
        if hasattr(message, 'content') and message.content:
            msg_type = type(message).__name__
            print(f"  {j+1}. {msg_type}: {message.content[:100]}...")
        elif hasattr(message, 'tool_calls') and message.tool_calls:
            print(f"  {j+1}. Tool Calls: {len(message.tool_calls)} tools called")
print("\n" + "=" * 60)

  1. SystemMessage: You are a helpful assistant with access to specialized tools. 

IMPORTANT RULES:
1. You MUST use the...
  2. HumanMessage: What's 25 multiplied by 4, then add 17 to the result?...
  3. Tool Calls: 1 tools called
  4. ToolMessage: 100...
  5. Tool Calls: 1 tools called
  6. ToolMessage: 117...
  7. AIMessage: The final answer is 117....



In [22]:
print("\n🎯 Detailed Example with Step Tracking:")
print("=" * 60)



🎯 Detailed Example with Step Tracking:


In [23]:
detailed_response = react_graph.invoke({
    "messages": [
        SystemMessage(content=SYSTEM_MESSAGE + "\n\nFor this task, explain each step you're taking and why you're using each tool."),
        HumanMessage(content="Find out when the Berlin Wall fell. Then calculate: (year × 3) - 2000 + 15")
    ]
})

/tmp/ipython-input-133929757.py:5: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  search = DDGS()
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e0591783f50>


🔢 Multiplication Tool: 1989 × 3 = 5967


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e0591964050>


➖ Subtraction Tool: 5967 - 2000 = 3967


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e05919640c0>


➕ Addition Tool: 3967 + 15 = 3982


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e0591964360>


In [24]:
print("\n📝 Complete Message History:")
for i, msg in enumerate(detailed_response["messages"]):
    print(f"\n--- Message {i+1} ---")
    msg.pretty_print()


📝 Complete Message History:

--- Message 1 ---
================================ System Message ================================

You are a helpful assistant with access to specialized tools. 

IMPORTANT RULES:
1. You MUST use the provided tools for ALL calculations - never perform math manually
2. For ANY arithmetic operation (addition, subtraction, multiplication), use the corresponding tool
3. For historical information or dates, use the search_historical_info tool
4. Always use tools even for simple calculations that you could do yourself
5. If you need the current year, use the get_current_year tool
6. Show your work by using tools step by step

Available tools:
- search_historical_info: For historical queries and dates
- perform_multiplication: For multiplication (REQUIRED for all multiplication)
- perform_addition: For addition (REQUIRED for all addition)
- perform_subtraction: For subtraction (REQUIRED for all subtraction)
- get_current_year: To get the current year
- calculate